In [3]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib as mp
import sklearn
import networkx as nx
from IPython.display import Image, HTML

import laUtilities as ut

%matplotlib inline

# Gradient Descent

Most of the machine learning we have studied this semester is based on the idea that we have a model is that _parameterized_, and our goal is to find good settings for the parameters.

We have seen example after example of this problem.

* In $k$-means, our goal was to find $k$ cluster centroids, so that the $k$-means objective was minimized.

* In linear regression, our goal was to find a parameter vector $\beta$ so that sum of squared error $\Vert \mathbf{y} - \hat{\mathbf{y}}\Vert$ was minimized.

* In the support vector machine, our goal was to find a parameter vector $\theta$ so that classification error was minimized.

And on and on ...

It's time now to talk about how, in general, one can find "good settings" for the parameters in problems like these.

What allows us to unify our approach to many such problems is the following:

First, we start by defining an error function, generally called a __loss__ function, to describe how well our method is doing.

And second, we choose loss functions that are __differentiable__ with respect to the parameters.

These two requirements mean that we can think of the parameter tuning problem using these sorts of surfaces.

<center>
    
<img src="figs/L23-convex_cost_function.jpeg" width="75%">
    
</center> 

Imagine that the $x$ and $y$ axes in these pictures represent parameter settings.   That is, we have two parameters to set, corresponding to the values of $x$ and $y$.

For each $(x, y)$ setting, the $z$-axis shows the value of the loss function. 

What we want to do is find the minimum of a surface, corresponding to the parameter settings that minimize loss.

Notice the difference between the two kinds of surfaces.    

The surface on the left corresponds to a __strictly convex__ loss function.   If we find a local minimum of this function, it is a global minimum.

The surface on the right corresponds to a __non-convex__ loss function.   There are local minima that are not globally minimal.

Both kinds of loss functions arise in machine learning.

For example, convex loss functions arise in
* Linear regression
* Logistic regression

While non-convex loss functions arise in 
* $k$-means
* Gaussian Mixture Modeling
* and many other settings

## Gradient Descent Intuitively

The intuition of gradient descent is the following.   

Imagine you are lost in the mountains, and it is foggy out.  You want to find a valley.  But since it is foggy, you can only see the local area around you.

<!-- Image credit http://nederlandliving.com/?p=1931 -->

<center>
    
<img src="figs/L23-fog-in-the-mountains.jpeg" width="60%">
    
</center> 

The natural thing to do is: 
1. Look around you 360 degrees.  
2. Observe in which direction the ground is sloping downward most steeply.  
3. Take a few steps in that direction.  
4. Repeat the process <BR> ... until the ground seems to be level.

## Formalizing Gradient Descent

The key to this intuitive idea is formalizing the idea of "direction of steepest descent."

This is where the differentiability of the loss function comes into play.

As long as the loss function is differentiable, we can define the direction of steepest descent (really, ascent).

That direction is called the __gradient.__

The gradient is a generalization of the slope of a line.

Let's say we have a loss function $\mathcal{L}(\mathbf{w})$.   

The components of $\mathbf{w}\in\mathbb{R}^n$ are the parameters we want to optimize.

To find the gradient, we take the partial derivative of our loss function with respect to each parameter:

$$ \frac{\partial \mathcal{L}}{\partial w_i} $$

and collect all the partial derivatives into a vector of the same shape as $\mathbf{w}$:

$$ \nabla_\mathbf{w}\mathcal{L} = \begin{bmatrix}
    \frac{\partial \mathcal{L}}{\partial w_1}\\
    \frac{\partial \mathcal{L}}{\partial w_2}\\
    \vdots \\
    \frac{\partial \mathcal{L}}{\partial w_n}
   \end{bmatrix}
   $$

When you see the notation  $\nabla_\mathbf{w}\mathcal{L},$ think of it as $ \frac{d\mathcal{L}}{d\mathbf{w}} $, except that $\mathbf{w}$ is a vector.

It turns out that if we are going to take a small step of unit length, then the gradient is the direction that maximizes the change in the loss function.

<!-- Image credit https://www.analyticsvidhya.com/blog/2020/10/how-does-the-gradient-descent-algorithm-work-in-machine-learning/ -->

<center>
    
<img src="figs/L23-gradient-of-convex.png" width="60%">
    
</center> 

As you can see from the above figure, in general the gradient varies depending on where you are in the parameter space.

So we write:

$$ \nabla_\mathbf{w}\mathcal{L}(\mathbf{w}) = \begin{bmatrix}
    \frac{\partial \mathcal{L}}{\partial w_1}(\mathbf{w})\\
    \frac{\partial \mathcal{L}}{\partial w_2}(\mathbf{w})\\
    \vdots \\
    \frac{\partial \mathcal{L}}{\partial w_n}(\mathbf{w})
   \end{bmatrix}
   $$

Each time we seek to improve our parameter estimates $\mathbf{w}$, we will take a step in the opposite direction of the gradient.

"opposite direction" because the gradient specifies the direction of maximum increase, and we want to decrease, the loss function.

How long should our step be?  

For step size, will use a scalar value $\eta$ called the __learning rate.__

The learning rate is a hyperparameter that needs to be tuned for a given problem, or even can be modified adaptively as the algorithm progresses.

Now we can write the __gradient descent__ algorithm formally:

1. Start with an initial parameter estimate $\mathbf{w}^0$.
2. Update: $\mathbf{w}^{n+1} = \mathbf{w}^n - \eta \nabla_\mathbf{w}\mathcal{L}(\mathbf{w}^n)$
3. If not converged, go to step 2.

How do we know if we are "converged"?  

Typically we stop the iteration if the loss has not improved by a fixed amount for a pre-decided number, say 10 or 20, iterations.

Here is an animation of the sequence of points on the loss surface explored by gradient descent:

<!-- https://blog.paperspace.com/intro-to-optimization-in-deep-learning-gradient-descent/ -->

<center>
    
<img src="figs/L23-gradient-animation.gif" width="60%">
    
</center> 

Notice that the improvement in loss decreases over time.  Initially the gradient is steep and loss improves fast, while later on the gradient is shallow and loss doesn't improve much per step.

<!-- https://www.cs.umd.edu/~tomg/projects/landscapes/ -->

<center>
    
<img src="figs/L23-complex-landscape.png" width="40%">
    
</center> 

For example. the likelihood function for logistic regression is:

define $\sigma(x) = \text{logit}^{-1}(x) = \frac{1}{1+e^{-x}} $

$$L(\alpha, \beta \mid x_i, y_i) = \sigma(\alpha + \beta x_i)^{y_i} (1-\sigma(\alpha + \beta x_i))^{1-y_i}$$

or more generally:


$$L(\alpha, \beta) = \sum_i \sigma(\alpha + \beta x_i)^{y_i} (1-\sigma(\alpha + \beta x_i))^{1-y_i}$$

We'd like to maximize the likelihood.  We'll take the log of it for convenience.

$$\log L(\alpha, \beta) = \sum_i \log(\sigma(\alpha + \beta x_i)^{y_i} (1-\sigma(\alpha + \beta x_i))^{1-y_i})$$

$$= \sum_i y_i \log(\sigma(\alpha + \beta x_i))+ (1-y_i) \log(1-\sigma(\alpha + \beta x_i))$$

To create a loss function that we can minimize, we negate the log likelihood.   Negative log likelihood is called __cross entropy__ loss.

$$\mathcal{L} = - \sum_i y_i \log(\sigma(\alpha + \beta x_i))+ (1-y_i) \log(1-\sigma(\alpha + \beta x_i))$$